# The JuMP ecosystem for mathematical optimization: Topics

## JuliaCon 2018

## Juan Pablo Vielma
## MIT Sloan

In [ ]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver
using Compat

# Duality

In [ ]:
model = Model(with_optimizer(GLPK.Optimizer))
@variable(model, x >= 0)
@variable(model, y >= 0)
@constraint(model,inequality, x + y <= 1)         
@objective(model, Max, x + 2y)
JuMP.optimize(model)
@show JuMP.terminationstatus(model) == MOI.Success
@show JuMP.dualstatus(model) == MOI.FeasiblePoint
@show JuMP.resultdual(inequality)

Also duals for variable bounds.

In [ ]:
@show JuMP.resultdual(JuMP.LowerBoundRef(x))
@show JuMP.resultdual(JuMP.LowerBoundRef(y))

For sign conventions and precise definition of "dual" problem see [Duality](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual.html#Duals-1) in [MOI Manual](http://www.juliaopt.org/MathOptInterface.jl/stable)

# More on JuMP Containers

In [ ]:
model = Model(with_optimizer(GLPK.Optimizer))
@variable(model, x[i=1:5, j=1:5])                        # Array      <=> @variable(m, x[i=1:5, j=1:5], container = Auto)
set_1 = Base.OneTo(5)
@variable(model, y[i=set_1, j=1:5])                      # Array
set_2 = 1:5
@variable(model, z1[i=1:5, j=set_2])                     # JuMPArray
@variable(model, z2[i=1:5, j=set_2], container = Array)  # Array
a = 1
@variable(model, zz1[i=1:5, j=a:5])                      # JuMPArray
@variable(model, zz2[i=1:5, j=a:5], container = Array)   # Array
set_3 = [:a, :b, :c]
@variable(model, w[i=set_2, j=set_3])                    # JuMPArray
@variable(model, t[i=set_2, j=1:i])                      # Dict
@variable(model, h[i = 1:5; j=isodd(i)])                 # Dict 

Auto chooses the tightest applicable container based on compile-time information only and is **type stable**

### `x[i=1:5, j=1:5]`

In [ ]:
#@macroexpand @variable(m, x[i=1:5, j=1:5])  # Array
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(x[i=1:5, j=1:5]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

### `y[i=set_1, j=1:5], set_1 = Base.OneTo(5)`

In [ ]:
#@macroexpand @variable(m, y[i=set_1, j=1:5]) # Array
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(y[i=set_1, j=1:5]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

### `z[i=1:5, j=set_2], set_2 = 1:5`

In [ ]:
#@macroexpand @variable(m, z1[i=1:5, j=set_2])  # JuMPArray
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(z[i=1:5, j=set_2]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

In [ ]:
#@macroexpand @variable(m, z2[i=1:5, j=set_2], container = Array) # Array  
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(z[i=1:5, j=set_2]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Array)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

In [ ]:
@code_typed JuMP.validarrayindexset(set_2)

In [ ]:
methods(JuMP.validarrayindexset)

In [ ]:
@code_typed JuMP.validarrayindexset(set_1)

### `zz[i=1:5, j=a:5], a = 1`

In [ ]:
#@macroexpand @variable(m, zz1[i=1:5, j=a:5])  # JuMPArray
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(zz1[i=1:5, j=a:5]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

In [ ]:
#@macroexpand @variable(m, zz1[i=1:5, j=a:5], container = Array)   # Array
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(zz1[i=1:5, j=a:5]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Array)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

In [ ]:
@code_typed JuMP.validarrayindexset(a:5)

### Warning: Testing with JuMP.validarrayindexset may **NOT** be **type stable**

In [ ]:
function getmyvariables(m,set)
    @variable(m,[set])
end

In [ ]:
function getmyvariableswithtesting(m,set)
    if JuMP.validarrayindexset(set)
        return @variable(m,[set],container=Array)
    else
        return @variable(m,[set])
    end
end

In [ ]:
tempbuff = IOBuffer()
code_warntype(tempbuff,getmyvariableswithtesting,Tuple{typeof(m),typeof(set_1)})
seekstart(tempbuff)
print(readlines(tempbuff)[end])

In [ ]:
tempbuff = IOBuffer()
code_warntype(tempbuff,getmyvariableswithtesting,Tuple{typeof(m),typeof(set_2)})
seekstart(tempbuff)
print(readlines(tempbuff)[end])

In [ ]:
tempbuff = IOBuffer()
code_warntype(tempbuff,getmyvariables,Tuple{typeof(m),typeof(set_2)})
seekstart(tempbuff)
print(readlines(tempbuff)[end])

**Note:** `@variable` should always be type stable: 

In [ ]:
function getmyvariablesarray(m,set)
    @variable(m,[set], container=Array)
end

In [ ]:
tempbuff = IOBuffer()
code_warntype(tempbuff,getmyvariablesarray,Tuple{typeof(m),typeof(set_1)})
seekstart(tempbuff)
print(readlines(tempbuff)[end])

In [ ]:
tempbuff = IOBuffer()
code_warntype(tempbuff,getmyvariablesarray,Tuple{typeof(m),typeof(set_2)})
seekstart(tempbuff)
print(readlines(tempbuff)[end])

In [ ]:
tempbuff = IOBuffer()
code_warntype(tempbuff,getmyvariablesarray,Tuple{typeof(m),typeof(set_3)})
seekstart(tempbuff)
print(readlines(tempbuff)[end])

In [ ]:
@code_warntype getmyvariablesarray(m,set_3)

# More Type Instability Warnings

cf. https://github.com/JuliaOpt/JuMP.jl/pull/1348

In [ ]:
using ECOS
model = Model(with_optimizer(ECOS.ECOSOptimizer))
@variable(model, x[i=1:5,j=1:5]) 

In [ ]:
@constraint(m, x in MOI.SecondOrderCone(5))

In [ ]:
function addmyconstraints(model,x)
    n,m = size(x)
    @constraint(model,[i=1:n], x[i,:] in MOI.SecondOrderCone(m))
end

In [ ]:
addmyconstraints(model,x)

In [ ]:
@code_warntype addmyconstraints(model,x)

In [ ]:
function addmyconstraintstype(model,x,consttype)
    n,m = size(x)
    constraints = (x -> x ? MOI.SecondOrderCone(m) : MOI.RotatedSecondOrderCone(m)).(consttype)
    @constraint(model,[i=1:n], x[i,:] in constraints[i])
end

In [ ]:
consttype = rand([true,false],5)

In [ ]:
@code_warntype addmyconstraintstype(model,x,consttype)